In [1]:
import torch
from transformer_engine import pytorch as te
from transformer_engine.common import recipe

In [2]:
in_features = 768
out_features = 3072
hidden_size = 2048

model = te.Linear(in_features, out_features, bias=True)
inp = torch.randn(hidden_size, in_features, device="cuda")

In [3]:
optim = torch.optim.AdamW((p for p in model.parameters() if p.requires_grad))

In [4]:
dir(recipe)

['Callable',
 'DelayedScaling',
 'Enum',
 'Float8BlockScaling',
 'Float8CurrentScaling',
 'Format',
 'Literal',
 'MMParams',
 'MXFP8BlockScaling',
 'NVFP4BlockScaling',
 'NamedTuple',
 'Optional',
 'QParams',
 'Recipe',
 'Union',
 '_FormatHelper',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'annotations',
 'dataclass',
 'os']

In [9]:
fp8_recipe = recipe.NVFP4BlockScaling()

with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
    with te.fp8_autocast(enabled=True, fp8_recipe=fp8_recipe):
        out = model(inp.to(torch.bfloat16))

In [10]:
optim.zero_grad()
loss = out.sum()
loss.backward()
optim.step()

In [11]:
loss

tensor(69.5000, device='cuda:0', dtype=torch.bfloat16, grad_fn=<SumBackward0>)